In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [58]:
df = pd.read_csv('dd/resultats/task_1_8/graph.txt', delim_whitespace=True, header=0)
list_2 = df['RHO_TOT'].values.tolist()
list_1 = df['TAU'].values.tolist()
array = np.column_stack([list_1[5:], list_2[5:]])
print(array)

[[4.8750e+01 5.5366e+09]
 [4.9500e+01 5.6049e+09]
 [5.0250e+01 5.6049e+09]
 [5.1000e+01 5.6049e+09]
 [5.1750e+01 5.7914e+09]
 [5.2500e+01 5.7914e+09]
 [5.3250e+01 5.7914e+09]
 [5.4000e+01 5.8597e+09]
 [5.4750e+01 5.8597e+09]
 [5.5500e+01 6.0462e+09]
 [5.6250e+01 6.0462e+09]
 [5.7000e+01 6.0462e+09]
 [5.7750e+01 6.0803e+09]
 [5.8500e+01 6.1144e+09]
 [5.9250e+01 6.1144e+09]
 [6.0000e+01 6.3009e+09]
 [6.0750e+01 6.3009e+09]
 [6.1500e+01 6.1144e+09]
 [6.2250e+01 6.1144e+09]
 [6.3000e+01 6.3692e+09]
 [6.3750e+01 6.3692e+09]
 [6.4500e+01 6.5557e+09]
 [6.5250e+01 6.3692e+09]
 [6.6000e+01 6.6240e+09]
 [6.6750e+01 6.6240e+09]
 [6.7500e+01 6.6240e+09]
 [6.8250e+01 6.6240e+09]
 [6.9000e+01 6.6240e+09]
 [6.9750e+01 6.8787e+09]
 [7.0500e+01 6.8787e+09]
 [7.1250e+01 6.8787e+09]
 [7.2000e+01 7.0652e+09]
 [7.2750e+01 7.0652e+09]
 [7.3500e+01 7.3541e+09]
 [7.4250e+01 9.1034e+09]
 [7.5000e+01 1.1396e+10]
 [7.5750e+01 1.4012e+10]
 [7.6500e+01 1.7188e+10]
 [7.7250e+01 2.0975e+10]
 [7.8000e+01 2.5051e+10]


In [45]:
def find_elbow(data, theta):

    # make rotation matrix
    co = np.cos(theta)
    si = np.sin(theta)
    rotation_matrix = np.array(((co, -si), (si, co)))

    # rotate data vector
    rotated_vector = data.dot(rotation_matrix)

    # return index of elbow
    return np.where(rotated_vector == rotated_vector[:, 1].min())[0][0]
    
def get_data_radiant(data):
  return np.arctan2(data[:, 1].max() - data[:, 1].min(), 
                    data[:, 0].max() - data[:, 0].min())
